# 1. Data preparation
Transfer data from .json to .csv and seperate training data and testing data

In [6]:
import pandas as pd
import os 
if ('train_df.csv' not in os.listdir('../kaggle_data')) or ('test_df.csv' not in os.listdir('../kaggle_data')):
    print("no file")

    data_id = pd.read_csv("../kaggle_data/data_identification.csv")
    emotion = pd.read_csv("../kaggle_data/emotion.csv")
    raw_df = pd.read_json("../kaggle_data/tweets_DM.json", lines=True, orient='columns')

    total_df = pd.json_normalize(raw_df._source)
    total_df = total_df.rename(columns={"tweet.tweet_id":"tweet_id", "tweet.hashtags":"hashtags", "tweet.text":"text"}
                              ).merge(data_id, on="tweet_id", how="left")
    total_df.to_csv('../kaggle_data/total_df.csv',index=False)

    train_df = total_df[total_df["identification"] == "train"]
    train_df = train_df.merge(emotion, on="tweet_id", how="left")
    test_df = total_df[total_df["identification"] == "test"]

    train_df.to_csv('../kaggle_data/train_df.csv',index=False)
    test_df.to_csv('../kaggle_data/test_df.csv',index=False) 

%reset -f

In [7]:
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer

Read training data and testing data

In [8]:
train_df = pd.read_csv("../kaggle_data/train_df.csv", lineterminator="\n")
test_df = pd.read_csv("../kaggle_data/test_df.csv", lineterminator="\n")

remove special characters, ex: "#", "@"

In [9]:
# Remove special chars that will affect results
# total_df['text'] = total_df['text'].str.replace(' ', '')
train_df['text'] = train_df['text'].str.replace('#', '').str.replace('@', '')
test_df['text'] = test_df['text'].str.replace('#', '').str.replace('@', '')
train_df.head()


,hashtags,tweet_id,text,identification,emotion
0,['Snapchat'],0x376b20,"People who post ""add me on Snapchat"" must be d...",train,anticipation
1,"['freepress', 'TrumpLegacy', 'CNN']",0x2d5350,"brianklaas As we see, Trump is dangerous to fr...",train,sadness
2,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,"['authentic', 'LaughOutLoud']",0x1d755c,RISKshow TheKevinAllison Thx for the BEST TIME...,train,joy
4,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation


cause i don't have the right answers of test data, i seperate the training data into training and validation data.

In [10]:
percent = 0.9
train_df = train_df.sample(frac=1)
T_len = round(train_df.shape[0] * percent)
Train_df = train_df[:T_len]
val_df = train_df[T_len:]

# 2. Feature engineering

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(preserve_case=False)

I choose tfidf vectorizer and the tweet tokenizer

In [12]:
# BOW_500 = CountVectorizer(max_features=10, tokenizer=nltk.word_tokenize) 
Tfidf_10000 = TfidfVectorizer(max_features=10000, stop_words='english', tokenizer=tknzr.tokenize)
Tfidf_10000.fit(train_df['text'])

X_train = Tfidf_10000.transform(Train_df['text'])
y_train = Train_df['emotion']
X_val = Tfidf_10000.transform(val_df['text'])
y_val = val_df['emotion']
X_test = Tfidf_10000.transform(test_df['text'])

/home/neaf2070/code/env/DM_new/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# 3. Model and Results Evaluation

Choose rogistic regression model

In [15]:
from sklearn import preprocessing, linear_model
LR_model = linear_model.LogisticRegression()

y_train = Train_df['emotion']
## training!
LR_model = LR_model.fit(X_train, y_train)

## predict!
y_train_pred = LR_model.predict(X_train)
y_val_pred = LR_model.predict(X_val)
y_test_pred = LR_model.predict(X_test)

/home/neaf2070/code/env/DM_new/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


check the accuracy of training data and validation data

In [16]:
## accuracy
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_val = accuracy_score(y_true=y_val, y_pred=y_val_pred)

print('training accuracy: {}'.format(round(acc_train, 6)))
print('val accuracy: {}'.format(round(acc_val, 6)))

training accuracy: 0.540437
val accuracy: 0.536


---
this is another model, but the performance is worse than LogisticRegression

In [17]:
## build MultinomialNB model
from sklearn.naive_bayes import MultinomialNB
NB_model = MultinomialNB()

## training!
NB_model = NB_model.fit(X_train, y_train)

## predict!
y_train_pred = NB_model.predict(X_train)
y_val_pred = NB_model.predict(X_val)
y_test_pred = NB_model.predict(X_test)

In [12]:
## build MultinomialNB model
NB_model = MultinomialNB()

## training!
NB_model = NB_model.fit(X_train, y_train)

## predict!
y_train_pred = NB_model.predict(X_train)
y_test_pred = NB_model.predict(X_test)

## so we get the pred result
y_test_pred[:10]

array(['anticipation', 'disgust', 'disgust', 'anticipation',
       'anticipation', 'sadness', 'sadness', 'joy', 'joy', 'joy'],
      dtype='<U12')

In [17]:
## accuracy
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
# acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 5)))
# print('testing accuracy: {}'.format(round(acc_test, 2)))


training accuracy: 0.44736


# 4. Save the result 
save the predicted label to .csv

In [17]:
yy = pd.DataFrame(y_test_pred)
output_test = pd.concat([test_df.tweet_id, yy], axis=1)
output_test = output_test.rename(columns={"tweet_id":"id", 0:"emotion"})
output_test.head()

,id,emotion
0,0x28b412,anticipation
1,0x2de201,anticipation
2,0x218443,disgust
3,0x2939d5,joy
4,0x26289a,anticipation


In [18]:
output_test.to_csv('../kaggle_data/output_test54_tweet_to_10000_tfidf_logic.csv',index=False)

# 5. Conclusion

1. The dataset is so big that I have to transfer it to a proper file type, in order to reuse it again. 
2. I choose the set of TF-IDF vectorizer and the tweet tokenizer because the result is much better than the set of BOW vectorizer and normal tokenizer. 
3. The performance of the logic regression model is better than the Naive Bayes model.